# Proyecto Bimestral: Sistema de Recuperaci´on de Información basado en Reuters-21578
## 1. Introducción
El objetivo de este proyecto es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578.
## 2. Fases del Proyecto
###  2.1. Adquisición de datos

*   Descargar el Corpus Reuters-21578
*   Descomprimir y organizar los archivos
*   Documentar el proceso de adquisición de datos

![Descripción de la imagen](images/corpus_reuters.jpg)

#### Descarga y análisis inicial del corpus

1. El archivo comprimido fue descargado y descomprimido en una carpeta local vinculada a un repositorio de GitHub.
2. Posteriormente, se analizó el contenido del corpus, obteniendo los siguientes resultados:
   - **Carpeta `test`**: Contiene 3019 archivos.
   - **Carpeta `training`**: Contiene 7769 archivos.
   - **Archivo `cats`**: Incluye las categorías.
   - **Archivo `readme`**: Proporciona información general sobre el corpus.
   - **Archivo `stopwords`**: Contiene una lista de palabras vacías.

#### Nota
Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install rarfile` para tener la biblioteca `rarfile` en nuestro entorno.

In [27]:
#!pip install rarfile

#### Nota
Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install nltk` para tener la biblioteca `nltk` en nuestro entorno.

In [29]:
#!pip install nltk

In [30]:
# Librerias necesarias
import os
import re
import nltk
import rarfile
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### 2.2. Preprocesamiento

#### 2.2.1. Extraer el contenido relevante de los documentos

In [19]:
rarfile.UNRAR_TOOL = r"D:\UnRAR\UnRAR.exe" # Cambiar por ubicación local de tu herramienta UNRAR

# Ruta del archivo .rar
rar_path = 'material/reuters.rar'
output_dir = 'material/content'

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Descomprimir el archivo .rar
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(output_dir)

# Verificar que se haya descomprimido correctamente
print("Archivos extraídos:")
print(os.listdir(output_dir))

Archivos extraídos:
['reuters']


In [21]:
# Directorios de documentos
train_dir = 'material/content/reuters/training'
test_dir = 'material/content/reuters/test'
cats_file = 'material/content/reuters/cats.txt'

# Diccionario para almacenar documentos
documentos = {}

# Función para extraer contenido de un archivo de noticias
def extraer_texto(filepath):
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            contenido = file.read()
            texto_limpio = contenido.strip()  # Elimina espacios en blanco iniciales y finales
            return texto_limpio
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return ""

# Función para cargar las categorías de los documentos
def cargar_categorias(filepath):
    categorias = {}
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            for linea in file:
                partes = linea.strip().split()  # Divide la línea en partes separadas por espacios
                if len(partes) >= 2:
                    doc_id = partes[0]  # Primer elemento es el id del documento (test/14826, training/1)
                    etiquetas = partes[1:]  # Resto de elementos son categorías
                    categorias[doc_id] = etiquetas
    except Exception as e:
        print(f"Error al leer el archivo de categorías {filepath}: {e}")
    return categorias

# Función para cargar los documentos y asociar categorías
def cargar_documentos(directorio, tipo, categorias_dict):
    archivos = os.listdir(directorio)
    if not archivos:
        print(f"No se encontraron archivos en {directorio}")
    for archivo in archivos:
        filepath = os.path.join(directorio, archivo)
        doc_id = f"{tipo}/{archivo}"
        texto = extraer_texto(filepath)
        categorias = categorias_dict.get(doc_id, [])  # Obtener categorías, si no hay, devuelve lista vacía

        # Almacenar en el diccionario
        documentos[doc_id] = {
            "texto": texto,
            "categorias": categorias
        }

In [22]:
# Cargar categorías
categorias_dict = cargar_categorias(cats_file)

# Cargar documentos de entrenamiento y prueba
cargar_documentos(train_dir, 'training', categorias_dict)
cargar_documentos(test_dir, 'test', categorias_dict)

# Verificar el tamaño y algunas muestras
print(f"Total de documentos cargados: {len(documentos)}")
if documentos:
    ejemplo_doc = list(documentos.items())[0]
    print(f"ID: {ejemplo_doc[0]}")
    print(f"Texto: {ejemplo_doc[1]['texto'][:500]}...")  # Mostrar los primeros 500 caracteres
    print(f"Categorías: {ejemplo_doc[1]['categorias']}")
else:
    print("No se cargaron documentos.")

Total de documentos cargados: 10788
ID: training/1
Texto: BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at th...
Categorías: ['cocoa']


#### 2.2.2. Realizar limpieza de datos: eliminación de caracteres no deseados, normalización de texto, etc.

In [24]:
# Función para limpiar texto
def limpiar_texto(texto):
    # 1. Conversión a minúsculas
    texto = texto.lower()

    # 2. Eliminación de caracteres especiales y números
    texto = re.sub(r'[^a-z\s]', '', texto)

    # 3. Eliminación de espacios extra
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

In [25]:
# Aplicar limpieza de texto a todos los documentos
for doc_id in documentos:
    texto_original = documentos[doc_id]['texto']
    texto_limpio = limpiar_texto(texto_original)

    # Actualizar el texto limpio en el diccionario
    documentos[doc_id]['texto'] = texto_limpio

# Verificar el resultado de la limpieza en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Texto limpio: {ejemplo_doc[1]['texto'][:500]}...")

ID: training/1
Texto limpio: bahia cocoa review showers continued throughout the week in the bahia cocoa zone alleviating the drought since early january and improving prospects for the coming temporao although normal humidity levels have not been restored comissaria smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february were bags of kilos making a cumulative total for the season of mln against at the same stage last year again it seems that cocoa delive...


#### 2.2.3. Tokenización: dividir el texto en palabras o tokens

In [33]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [26]:
# Función para tokenizar texto
def tokenizar_texto(texto):
    # Utilizamos word_tokenize de NLTK para dividir en tokens
    tokens = word_tokenize(texto)
    return tokens

In [34]:
# Aplicar tokenización a todos los documentos
for doc_id in documentos:
    texto_limpio = documentos[doc_id]['texto']
    tokens = tokenizar_texto(texto_limpio)

    # Guardamos los tokens en el diccionario
    documentos[doc_id]['tokens'] = tokens

# Verificar el resultado de la tokenización en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens: ['bahia', 'cocoa', 'review', 'showers', 'continued', 'throughout', 'the', 'week', 'in', 'the', 'bahia', 'cocoa', 'zone', 'alleviating', 'the', 'drought', 'since', 'early', 'january', 'and']...


#### 2.2.4. Eliminar stop words y  aplicar stemming o lematización.

In [38]:
# Cargar el archivo de stopwords proporcionado
ruta_stopwords = 'material/content/reuters/stopwords'
stopwords_personalizadas = set()

with open(ruta_stopwords, 'r') as archivo:
    for linea in archivo:
        palabra = linea.strip()  # Removemos espacios y saltos de línea
        if palabra:  # Evitamos añadir líneas vacías
            stopwords_personalizadas.add(palabra.lower())

# Verificamos algunas stop words cargadas
print("Ejemplo de Stop Words cargadas:", list(stopwords_personalizadas)[:10])

Ejemplo de Stop Words cargadas: ['rather', 'however', 'willing', 'q', 'secondly', 'but', 'specify', 'you', 'own', 'know']


In [42]:
# Función para eliminar stop words de los tokens
def eliminar_stopwords(tokens, stopwords):
    # Filtramos los tokens que no están en la lista de stopwords
    tokens_filtrados = [token for token in tokens if token.lower() not in stopwords]
    return tokens_filtrados

# Aplicamos la eliminación de stop words a cada documento
for doc_id in documentos:
    tokens = documentos[doc_id]['tokens']
    tokens_sin_stopwords = eliminar_stopwords(tokens, stopwords_personalizadas)

    # Guardamos los tokens filtrados
    documentos[doc_id]['tokens'] = tokens_sin_stopwords

# Verificar el resultado después de eliminar stop words en un documento de ejemplo
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens sin Stop Words: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens sin Stop Words: ['bahia', 'cocoa', 'review', 'showers', 'continued', 'week', 'bahia', 'cocoa', 'zone', 'alleviating', 'drought', 'early', 'january', 'improving', 'prospects', 'coming', 'temporao', 'normal', 'humidity', 'levels']...


#### Nota
Para nuestro caso de estudio y aplicación de este proyecto, aplicaremos **Steming**. 

Como equipo, consideramos más eficiente en términos de recursos porque el stemming utiliza reglas más simples y rápidas para reducir las palabras a su raíz, mientras que la lemmatización requiere un procesamiento más complejo y el uso de diccionarios para obtener la forma base correcta de las palabras.

In [43]:
# Inicializar el stemmer
stemmer = PorterStemmer()

# Función para aplicar stemming a los tokens de un documento
def aplicar_stemming(tokens):
    return [stemmer.stem(token) for token in tokens]

In [44]:
# Aplicamos el stemming a los tokens de cada documento
for doc_id in documentos:
    tokens = documentos[doc_id]['tokens']  # Obtiene los tokens del documento
    tokens_stemmed = aplicar_stemming(tokens)  # Aplica el stemming
    documentos[doc_id]['tokens'] = tokens_stemmed  # Guarda los tokens procesados

# Verificamos un documento de ejemplo después del stemming
ejemplo_doc = list(documentos.items())[0]
print(f"ID: {ejemplo_doc[0]}")
print(f"Tokens con Stemming: {ejemplo_doc[1]['tokens'][:20]}...")

ID: training/1
Tokens con Stemming: ['bahia', 'cocoa', 'review', 'shower', 'continu', 'week', 'bahia', 'cocoa', 'zone', 'allevi', 'drought', 'earli', 'januari', 'improv', 'prospect', 'come', 'temporao', 'normal', 'humid', 'level']...


### 2.3.  Representación de Datos en Espacio Vectorial